In [ ]:
import pandas as pd 
import numpy as np 
import scipy.stats
from decimal import Decimal 


Deleteriousness burden, 417 CRISPR hits with 10+ variants

Regeneron burden, up to 490 CRISPR hits (may be fewer depending on which genes have variants)

Deleteriousness burden, genome-wide

Regeneron burden, genome-wide

Deleteriousness burden, module 255 (up to 14 genes, may be fewer depending on which have variants)

Regeneron burden, module 255 (up to 14 genes, may be fewer depending on which have variants)

Deleteriousness burden, module 69 (54 genes, may be fewer depending on which have variants)

Regeneron burden, module 69 (54 genes, may be fewer depending on which have variants)

In [ ]:
def get_harmoic_p(p_list, weights = None):
    if weights is None:
        weights = [1 / len(p_list)] * len(p_list)
    numerator = np.sum(weights)
    denominator = 0
    for w,p in zip(weights, p_list):
        denominator += (w/p)
    return numerator / denominator 

def simes(p_vals):
    sorted_p = p_vals.sort_values()
    ranks = pd.Series(range(1, len(p_vals) + 1))
    multiplied = sorted_p * len(ranks)
    results = multiplied/ranks.values
    return min(results)

def get_browns_chi_sq(p_list, dof):
    summation = 0
    for p in p_list:
        summation += np.exp(p)
    X = -2 * summation 
    print(X)
    return scipy.stats.chi2.cdf(X, dof)

def format_p(p):
    if p < 0.0001:
        sci_notation_pieces = "{:.2e}".format(p).split("e")
        exponent = int(sci_notation_pieces[1])
        decimal = sci_notation_pieces[0]
        p_string =  decimal + " x 10^{" + str(exponent) + "}" +  "$"
        return p_string
    else:
        return round(p , 3)

In [ ]:
with open("490_genes_from_cell_screen.txt", "r") as f:
    genes_490 = list(map(lambda x : x.replace("\n", ""), f.readlines()))
f.close()

with open("LDL_inc_genes_from_screen.txt", "r") as f:
    genes_increase = list(map(lambda x : x.replace("\n", ""), f.readlines()))
f.close()

with open("LDL_dec_genes_from_screen.txt", "r") as f:
    genes_decrease = list(map(lambda x : x.replace("\n", ""), f.readlines()))
f.close()

df = pd.read_csv("./GSEA_summaries/perc_25_d_2_100k_iterations.csv")

module_69_genes = df.loc[df["Term"] == "module_69"].iloc[0]["genes"].split(";")
module_255_genes = df.loc[df["Term"] == "module_255"].iloc[0]["genes"].split(";")

all_vals = []

### Deleteriousness burden, 417 CRISPR hits with 10+ variants

In [ ]:
mega_df = pd.concat([
    pd.read_csv('./alternate_score_files/042722_whole_genome_0th_perc_442_genes.csv'),
    pd.read_csv(f"041522_whole_genome_25th_perc.csv"),
    pd.read_csv(f"041522_whole_genome_50th_perc.csv"),
    pd.read_csv(f"041522_whole_genome_75th_perc.csv")
])

df = mega_df

print(len(df))

df["mwu_a_pval"] = df[['mwu_a_pval_greater', 'mwu_a_pval_less']].min(axis=1)
df["mwu_two_sided"] = df["mwu_a_pval"].apply(lambda x : x*2)
df["mwu_a_pval_intra"] = df[['mwu_a_pval_intra_greater','mwu_a_pval_intra_less']].min(axis=1)

col_of_interest = "mwu_a_pval"

subset = df.loc[
    (df["gene"].isin(genes_490)) &
    (~df[col_of_interest].isna()) &
    (df['high_impact_carriers'] >= 10)
].sort_values(by = col_of_interest)

alt_values = []
for g in set(subset["gene"].values):
    s = subset.loc[subset["gene"] == g]
    p_vals = pd.Series(s[col_of_interest].values)
    simes_p = simes(p_vals)
    harmonic_p = get_harmoic_p(p_vals.values)
    alt_values.append([g, simes_p, harmonic_p])
    
simes_df = pd.DataFrame(alt_values, columns = ["gene", "simes_p", "harmonic_p"])
m = len(simes_df)
print(m)
print("------")

signif = set()
alpha = 0.1
simes_df = simes_df.sort_values(by = "simes_p")
for counter, (index, row) in enumerate(simes_df.iterrows()):
    if row["simes_p"] <= ((counter+1) / m) * alpha:
        signif.add(row["gene"])
    
print(len(signif))
print(", ".join(list(signif)))

title = "Deleteriousness burden, 417 CRISPR hits with 10+ variant"
all_vals.append([title, m, len(signif), "|".join(sorted(list(signif)))])
print("less than 0.05", len(simes_df.loc[simes_df["simes_p"] <= 0.05]))
print( "|".join(sorted(list(signif))))


###  Regeneron burden, up to 490 CRISPR hits (may be fewer depending on which genes have variants)

In [ ]:
df = pd.read_csv("./data/GCST90083019_buildGRCh38.tsv", delimiter = "\t")
df["gene"] = df["Name"].apply(lambda x : x.split("(")[0])
df = df.loc[
    (df["gene"].isin(genes_490)) &
    (~df["p_value"].isna())
]
alt_values = []
for g in set(df["gene"].values):
    s = df.loc[df["gene"] == g]
    p_vals = pd.Series(s["p_value"].values)
    simes_p = simes(p_vals)
    harmonic_p = get_harmoic_p(p_vals.values)
    alt_values.append([g, simes_p, harmonic_p])
    
simes_df = pd.DataFrame(alt_values, columns = ["gene", "simes_p", "harmonic_p"])

signif = set()
alpha = 0.1
simes_df = simes_df.sort_values(by = "simes_p")
m = len(simes_df)

for counter, (index, row) in enumerate(simes_df.iterrows()):
    if row["simes_p"] <= ((counter+1) / m) * alpha:
        signif.add(row["gene"])

print(len(signif), m)
print(", ".join(sorted(list(signif))))
print("less than 0.05", len(simes_df.loc[simes_df["simes_p"] <= 0.05]))

title = "Regeneron burden, up to 490 CRISPR hits (may be fewer depending on which genes have variants)"
all_vals.append([title, m, len(signif), "|".join(sorted(list(signif)))])

### Deleteriousness burden, genome-wide

In [ ]:
mega_df = pd.concat([
    pd.read_csv('041522_whole_genome_0th_perc.csv'),
    pd.read_csv(f"041522_whole_genome_25th_perc.csv"),
    pd.read_csv(f"041522_whole_genome_50th_perc.csv"),
    pd.read_csv(f"041522_whole_genome_75th_perc.csv")
])

df = mega_df

print(len(df))

df["mwu_a_pval"] = df[['mwu_a_pval_greater', 'mwu_a_pval_less']].min(axis=1)
df["mwu_two_sided"] = df["mwu_a_pval"].apply(lambda x : x*2)
df["mwu_a_pval_intra"] = df[['mwu_a_pval_intra_greater','mwu_a_pval_intra_less']].min(axis=1)


col_of_interest = "mwu_a_pval"

subset = df.loc[
#     (df["gene"].isin(genes_490)) &
    (~df[col_of_interest].isna()) &
    (df['high_impact_carriers'] >= 10)
].sort_values(by = col_of_interest)

alt_values = []
for g in set(subset["gene"].values):
    s = subset.loc[subset["gene"] == g]
    p_vals = pd.Series(s[col_of_interest].values)
    simes_p = simes(p_vals)
    harmonic_p = get_harmoic_p(p_vals.values)
    alt_values.append([g, simes_p, harmonic_p])
    
simes_df = pd.DataFrame(alt_values, columns = ["gene", "simes_p", "harmonic_p"])
m = len(simes_df)

print("------")

signif = set()
alpha = 0.1
simes_df = simes_df.sort_values(by = "simes_p")
for counter, (index, row) in enumerate(simes_df.iterrows()):
    if row["simes_p"] <= ((counter+1) / m) * alpha:
        signif.add(row["gene"])
 


print(len(signif))
print(", ".join(list(signif)))
print("less than 0.05", len(simes_df.loc[simes_df["simes_p"] <= 0.05]))

title = "Deleteriousness burden, genome-wide"
all_vals.append([title, m, len(signif), "|".join(sorted(list(signif)))])

### Regeneron burden, genome-wide

In [ ]:
df = pd.read_csv("./data/GCST90083019_buildGRCh38.tsv", delimiter = "\t")
df["gene"] = df["Name"].apply(lambda x : x.split("(")[0])
df = df.loc[
    (~df["p_value"].isna())
]
alt_values = []
for g in set(df["gene"].values):
    s = df.loc[df["gene"] == g]
    p_vals = pd.Series(s["p_value"].values)
    simes_p = simes(p_vals)
    harmonic_p = get_harmoic_p(p_vals.values)
    alt_values.append([g, simes_p, harmonic_p])
    
simes_df = pd.DataFrame(alt_values, columns = ["gene", "simes_p", "harmonic_p"])

signif = set()
alpha = 0.1
simes_df = simes_df.sort_values(by = "simes_p")
m = len(simes_df)

for counter, (index, row) in enumerate(simes_df.iterrows()):
    if row["simes_p"] <= ((counter+1) / m) * alpha:
        signif.add(row["gene"])
    
    
print(len(signif), m - len(signif))
print(", ".join(sorted(list(signif))))
print("less than 0.05", len(simes_df.loc[simes_df["simes_p"] <= 0.05]))

title = "Regeneron burden, genome-wide"
all_vals.append([title, m, len(signif), "|".join(sorted(list(signif)))])

### Deleteriousness burden, module 255 (up to 14 genes, may be fewer depending on which have variants)

In [ ]:
mega_df = pd.concat([
    pd.read_csv('041522_whole_genome_0th_perc.csv'),
    pd.read_csv(f"041522_whole_genome_25th_perc.csv"),
    pd.read_csv(f"041522_whole_genome_50th_perc.csv"),
    pd.read_csv(f"041522_whole_genome_75th_perc.csv")
])

df = mega_df

df["mwu_a_pval"] = df[['mwu_a_pval_greater', 'mwu_a_pval_less']].min(axis=1)
df["mwu_two_sided"] = df["mwu_a_pval"].apply(lambda x : x*2)
df["mwu_a_pval_intra"] = df[['mwu_a_pval_intra_greater','mwu_a_pval_intra_less']].min(axis=1)

col_of_interest = "mwu_a_pval"

subset = df.loc[
    (df["gene"].isin(module_255_genes)) &
    (~df[col_of_interest].isna()) &
    (df['high_impact_carriers'] >= 10)
].sort_values(by = col_of_interest)

alt_values = []
for g in set(subset["gene"].values):
    s = subset.loc[subset["gene"] == g]
    p_vals = pd.Series(s[col_of_interest].values)
    simes_p = simes(p_vals)
    harmonic_p = get_harmoic_p(p_vals.values)
    alt_values.append([g, simes_p, harmonic_p])
    
simes_df = pd.DataFrame(alt_values, columns = ["gene", "simes_p", "harmonic_p"])
m = len(simes_df)
print(m)
print("------")

signif = set()
alpha = 0.1
simes_df = simes_df.sort_values(by = "simes_p")
for counter, (index, row) in enumerate(simes_df.iterrows()):
    if row["simes_p"] <= ((counter+1) / m) * alpha:
        signif.add(row["gene"])
    
print(len(signif))
print(", ".join(list(signif)))


print("less than 0.05", len(simes_df.loc[simes_df["simes_p"] <= 0.05]))

title = "Deleteriousness burden, module 255 (up to 14 genes, may be fewer depending on which have variants)"
all_vals.append([title, m, len(signif), "|".join(sorted(list(signif)))])

### Regeneron burden, module 255 (up to 14 genes, may be fewer depending on which have variants)

In [ ]:
df = pd.read_csv("./data/GCST90083019_buildGRCh38.tsv", delimiter = "\t")
df["gene"] = df["Name"].apply(lambda x : x.split("(")[0])
df = df.loc[
    (df["gene"].isin(module_255_genes)) &
    (~df["p_value"].isna())
]
alt_values = []
for g in set(df["gene"].values):
    s = df.loc[df["gene"] == g]
    p_vals = pd.Series(s["p_value"].values)
    simes_p = simes(p_vals)
    harmonic_p = get_harmoic_p(p_vals.values)
    alt_values.append([g, simes_p, harmonic_p])
    
simes_df = pd.DataFrame(alt_values, columns = ["gene", "simes_p", "harmonic_p"])

signif = set()
alpha = 0.1
simes_df = simes_df.sort_values(by = "simes_p")
m = len(simes_df)

for counter, (index, row) in enumerate(simes_df.iterrows()):
    if row["simes_p"] <= ((counter+1) / m) * alpha:
        signif.add(row["gene"])
    
print(len(signif), m - len(signif))
print(", ".join(sorted(list(signif))))

print("less than 0.05", len(simes_df.loc[simes_df["simes_p"] <= 0.05]))

title = "Regeneron burden, module 255 (up to 14 genes, may be fewer depending on which have variants)"
all_vals.append([title, m, len(signif), "|".join(sorted(list(signif)))])

### Deleteriousness burden, module 69 

In [ ]:
mega_df = pd.concat([
    pd.read_csv('041522_whole_genome_0th_perc.csv'),
    pd.read_csv(f"041522_whole_genome_25th_perc.csv"),
    pd.read_csv(f"041522_whole_genome_50th_perc.csv"),
    pd.read_csv(f"041522_whole_genome_75th_perc.csv")
])

df = mega_df

df["mwu_a_pval"] = df[['mwu_a_pval_greater', 'mwu_a_pval_less']].min(axis=1)
df["mwu_two_sided"] = df["mwu_a_pval"].apply(lambda x : x*2)
df["mwu_a_pval_intra"] = df[['mwu_a_pval_intra_greater','mwu_a_pval_intra_less']].min(axis=1)


col_of_interest = "mwu_a_pval"

subset = df.loc[
    (df["gene"].isin(module_69_genes)) &
    (~df[col_of_interest].isna()) &
    (df['high_impact_carriers'] >= 10)
].sort_values(by = col_of_interest)

alt_values = []
for g in set(subset["gene"].values):
    s = subset.loc[subset["gene"] == g]
    p_vals = pd.Series(s[col_of_interest].values)
    simes_p = simes(p_vals)
    harmonic_p = get_harmoic_p(p_vals.values)
    alt_values.append([g, simes_p, harmonic_p])
    
simes_df = pd.DataFrame(alt_values, columns = ["gene", "simes_p", "harmonic_p"])
m = len(simes_df)
print(m)
print("------")

signif = set()
alpha = 0.1
simes_df = simes_df.sort_values(by = "simes_p")
for counter, (index, row) in enumerate(simes_df.iterrows()):
    if row["simes_p"] <= ((counter+1) / m) * alpha:
        signif.add(row["gene"])
    
print(len(signif))
print(", ".join(list(signif)))

print("less than 0.05", len(simes_df.loc[simes_df["simes_p"] <= 0.05]))


title = "Deleteriousness burden, module 69"
all_vals.append([title, m, len(signif), "|".join(sorted(list(signif)))])


### Regeneron burden, module 69 

In [ ]:
df = pd.read_csv("./data/GCST90083019_buildGRCh38.tsv", delimiter = "\t")
df["gene"] = df["Name"].apply(lambda x : x.split("(")[0])
df = df.loc[
    (df["gene"].isin(module_69_genes)) &
    (~df["p_value"].isna())
]
alt_values = []
for g in set(df["gene"].values):
    s = df.loc[df["gene"] == g]
    p_vals = pd.Series(s["p_value"].values)
    simes_p = simes(p_vals)
    harmonic_p = get_harmoic_p(p_vals.values)
    alt_values.append([g, simes_p, harmonic_p])
    
simes_df = pd.DataFrame(alt_values, columns = ["gene", "simes_p", "harmonic_p"])

signif = set()
alpha = 0.1
simes_df = simes_df.sort_values(by = "simes_p")
m = len(simes_df)

for counter, (index, row) in enumerate(simes_df.iterrows()):
    if row["simes_p"] <= ((counter+1) / m) * alpha:
        signif.add(row["gene"])
    
print(len(signif), m - len(signif))
print(", ".join(sorted(list(signif))))
print("less than 0.05", len(simes_df.loc[simes_df["simes_p"] <= 0.05]))

title = "Regeneron burden, module 69"
all_vals.append([title, m, len(signif), "|".join(sorted(list(signif)))])

In [ ]:
df = pd.DataFrame(all_vals, columns = ["comparison", "n genes", "n signif", "significant genes"])

In [ ]:
df.to_csv("VEST4_Regeneron_significant_comparisons_8_conditions.csv")